In [2]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
import requests_cache
import lxml.html as lxl
import re
import sqlalchemy

In [3]:
MAX_PAGE = 333

In [4]:
## Get all the urls for all the listed used vehicles on truecar.com
def urls_scraping(base_url = 'https://www.truecar.com/used-cars-for-sale/listings/'):
    urls = []
    pages = []
    for i in range(1, MAX_PAGE+1):
        pages.append(base_url + '?page=' + str(i))
    for page in pages:
        try:
            response = requests.get(page)
            response.raise_for_status()
        except:
            break
        root = lxl.fromstring(response.content)
        url = ['https://www.truecar.com' + link for link in root.xpath('//div[@data-qa="Listings"]/a/@href')]
        urls += url
    
    return urls 

In [6]:
urls=urls_scraping()
urls

['https://www.truecar.com/used-cars-for-sale/listing/1FMCU0GD7HUE03208/2017-ford-escape/?zipcode=76131',
 'https://www.truecar.com/used-cars-for-sale/listing/1G11Z5S31KU106394/2019-chevrolet-impala/?sponsoredVehiclePosition=0',
 'https://www.truecar.com/used-cars-for-sale/listing/WBSEH93578CY24901/2008-bmw-m6/?sponsoredVehiclePosition=1',
 'https://www.truecar.com/used-cars-for-sale/listing/3FA6P0H77HR222432/2017-ford-fusion/',
 'https://www.truecar.com/used-cars-for-sale/listing/5NPD84LF9HH059516/2017-hyundai-elantra/',
 'https://www.truecar.com/used-cars-for-sale/listing/SHHFK7H26HU223996/2017-honda-civic/',
 'https://www.truecar.com/used-cars-for-sale/listing/1FTEW1CG5GFA17337/2016-ford-f-150/',
 'https://www.truecar.com/used-cars-for-sale/listing/KNMAT2MV7HP502570/2017-nissan-rogue/',
 'https://www.truecar.com/used-cars-for-sale/listing/1FTEW1EF4GKD46428/2016-ford-f-150/',
 'https://www.truecar.com/used-cars-for-sale/listing/3FA6P0H72HR157070/2017-ford-fusion/',
 'https://www.truec

In [56]:
# function to scrape one single url of a single used car listing.
def page_scraping(url):
    print(url)
    try:
        response = requests.get(url)
        response.raise_for_status()
    except:
        return
    root = lxl.fromstring(response.content)
    
    # extract vehicle year, make and model information
    year = root.xpath('//div[@class="text-truncate heading-3 margin-right-2 margin-right-sm-3"]/text()')[0].split(' ')[0]    
    make = root.xpath('//div[@class="text-truncate heading-3 margin-right-2 margin-right-sm-3"]/text()')[0].split(' ')[1]    
    model = root.xpath('//div[@class="text-truncate heading-3 margin-right-2 margin-right-sm-3"]/text()')[0].split(' ')[2:]    
    # need to extract vehicle type (suv or sedan) and sub-model info
    sub_model = root.xpath('//div[@class="text-truncate heading-4 text-muted"]/text()')[0] 

    # city and state, geospatial information.
    city = root.xpath('//span[@data-qa="used-vdp-header-location"]/text()[1]')[0]
    state = root.xpath('//span[@data-qa="used-vdp-header-location"]/text()[3]')[0]
    
    # vehicle mileage
    mileage = root.xpath('//span[@data-qa="used-vdp-header-miles"]/text()[1]')[0]
    
    # vehicle price information
    price = root.xpath('//html/body/div[2]/div[3]/div/div[4]/div/div[2]/div[1]/div/div/div[1]/div[3]/span/text()')[0]
    
    # vehicle characteristics
    exterior_color = root.xpath('//div[@data-qa="vehicle-overview-item-Exterior Color"]/div[2]/ul/li/text()')[0]
    interior_color = root.xpath('//div[@data-qa="vehicle-overview-item-Interior Color"]/div[2]/ul/li/text()')[0]
    mpg_city = root.xpath('//div[@data-qa="vehicle-overview-item-MPG"]/div[2]/ul/li/text()')[0].split('/')[0].split(' ')[0]
    mpg_hwy = root.xpath('//div[@data-qa="vehicle-overview-item-MPG"]/div[2]/ul/li/text()')[0].split('/')[1].split(' ')[1]
    engine = root.xpath('//div[@data-qa="vehicle-overview-item-Engine"]/div[2]/ul/li/text()')[0]
    transmission = root.xpath('//div[@data-qa="vehicle-overview-item-Transmission"]/div[2]/ul/li/text()')[0]
    drive_type = root.xpath('//div[@data-qa="vehicle-overview-item-Drive Type"]/div[2]/ul/li/text()')[0]
    fuel_type = root.xpath('//div[@data-qa="vehicle-overview-item-Fuel Type"]/div[2]/ul/li/text()')[0]
    popular_feature = root.xpath('//div[@data-test="popularFeatures"]//li[@class="_19zze7p"]/p/text()')
    
    # vehicle history information, will extract four variables from here.
    vehicle_history = root.xpath('//li[@class="_h9wfdq"]/text()')
    
    # whether the car is a certified preowned car.
    if "used-vdp-header-cpo" in response.text:
        cpo = True
    else:
        cpo = False
    
    
    return pd.Series({'year':year, 'make': make, 'model': model, 'sub_model': sub_model, 'city':city, 'state': state,
           'mileage': mileage, 'price': price, 'exterior_color': exterior_color,
                     'interior_color': interior_color, 'mpg_city': mpg_city, 'mpg_hwy': mpg_hwy, 'engine': engine,
                     'transmission': transmission, 'drive_type': drive_type, 'fuel_type': fuel_type, 
                     'popular_feature': popular_feature, 'vehicle_history': vehicle_history, 'cpo': cpo})

In [57]:
# Use multi-processing to speed up the web-scraping
# fully make use of 8 cores of my macbook pro.
from multiprocessing import Pool
num_partitions = 32
num_cores = 8
def parallelize(urls, func):
    url_set = np.array_split(urls, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, url_set))
    pool.close()
    pool.join()
    return df
def scraping(urls):
    scraping_data = [page_scraping(url) for url in urls]
    return pd.concat(scraping_data, axis=1).T

In [58]:
%%time
data = parallelize(urls, scraping)

https://www.truecar.com/used-cars-for-sale/listing/4T1B11HK4KU255663/2019-toyota-camry/
https://www.truecar.com/used-cars-for-sale/listing/3FA6P0HD3HR276767/2017-ford-fusion/?zipcode=76131
https://www.truecar.com/used-cars-for-sale/listing/5GTDN13E478214462/2007-hummer-h3/
https://www.truecar.com/used-cars-for-sale/listing/3FA6P0HD6HR244167/2017-ford-fusion/
https://www.truecar.com/used-cars-for-sale/listing/1C3CCCAB1FN617011/2015-chrysler-200/
https://www.truecar.com/used-cars-for-sale/listing/1GAZG1FG7B1187479/2011-chevrolet-express-passenger/
https://www.truecar.com/used-cars-for-sale/listing/1C6RR7TT3KS551193/2019-ram-1500-classic/
https://www.truecar.com/used-cars-for-sale/listing/3GYEK62N45G105519/2005-cadillac-escalade-ext/
https://www.truecar.com/used-cars-for-sale/listing/KNMAT2MT3HP545060/2017-nissan-rogue/
https://www.truecar.com/used-cars-for-sale/listing/JTEEP21A140061668/2004-toyota-highlander/
https://www.truecar.com/used-cars-for-sale/listing/1G1ZD5ST8KF113073/2019-chev

https://www.truecar.com/used-cars-for-sale/listing/JN8AE2KP8D9061274/2013-nissan-quest/
https://www.truecar.com/used-cars-for-sale/listing/2FMDK3JC2BBB12429/2011-ford-edge/
https://www.truecar.com/used-cars-for-sale/listing/1FMCU0GD0HUA36412/2017-ford-escape/
https://www.truecar.com/used-cars-for-sale/listing/JHMGK5H52GS009480/2016-honda-fit/
https://www.truecar.com/used-cars-for-sale/listing/1HGCS2B87CA007629/2012-honda-accord/
https://www.truecar.com/used-cars-for-sale/listing/JHMFB4F3XCS001919/2012-honda-civic/
https://www.truecar.com/used-cars-for-sale/listing/2D8HN54X98R692030/2008-dodge-grand-caravan/
https://www.truecar.com/used-cars-for-sale/listing/WP0CA2998XS650104/1999-porsche-911/
https://www.truecar.com/used-cars-for-sale/listing/1FMCU9GDXHUC38615/2017-ford-escape/
https://www.truecar.com/used-cars-for-sale/listing/4T1BF1FK4HU283072/2017-toyota-camry/
https://www.truecar.com/used-cars-for-sale/listing/1G1ZD5ST8KF161429/2019-chevrolet-malibu/
https://www.truecar.com/used-ca

https://www.truecar.com/used-cars-for-sale/listing/1FTEW1E56KFC33962/2019-ford-f-150/
https://www.truecar.com/used-cars-for-sale/listing/1HGCR3F01HA011429/2017-honda-accord/
https://www.truecar.com/used-cars-for-sale/listing/1FMCU0GXXDUB78710/2013-ford-escape/
https://www.truecar.com/used-cars-for-sale/listing/1FMCU0GD2HUC27894/2017-ford-escape/
https://www.truecar.com/used-cars-for-sale/listing/5YMGZ0C59E0C40371/2014-bmw-x6-m/
https://www.truecar.com/used-cars-for-sale/listing/2C4RDGCG8KR747884/2019-dodge-grand-caravan/
https://www.truecar.com/used-cars-for-sale/listing/2T1BURHE1JC995171/2018-toyota-corolla/
https://www.truecar.com/used-cars-for-sale/listing/1N4AL3AP3FC596808/2015-nissan-altima/
https://www.truecar.com/used-cars-for-sale/listing/5NPE24AF2HH531788/2017-hyundai-sonata/
https://www.truecar.com/used-cars-for-sale/listing/3FA6P0LU1KR121813/2019-ford-fusion/
https://www.truecar.com/used-cars-for-sale/listing/1VWCN7A34DC001184/2013-volkswagen-passat/
https://www.truecar.com/

https://www.truecar.com/used-cars-for-sale/listing/WA1LKBFP6AA097489/2010-audi-q5/
https://www.truecar.com/used-cars-for-sale/listing/1VWCN7A38CC032971/2012-volkswagen-passat/
https://www.truecar.com/used-cars-for-sale/listing/1FMCU9GD8HUC41058/2017-ford-escape/
https://www.truecar.com/used-cars-for-sale/listing/1G1ZD5ST9JF139034/2018-chevrolet-malibu/
https://www.truecar.com/used-cars-for-sale/listing/1FTEW1EF6HKC66212/2017-ford-f-150/
https://www.truecar.com/used-cars-for-sale/listing/1C6RR7LT4ES152152/2014-ram-1500/
https://www.truecar.com/used-cars-for-sale/listing/WDBSK71F77F134671/2007-mercedes-benz-sl/
https://www.truecar.com/used-cars-for-sale/listing/1FADP3K22GL255485/2016-ford-focus/
https://www.truecar.com/used-cars-for-sale/listing/1FTEW1EP8HKD75862/2017-ford-f-150/
https://www.truecar.com/used-cars-for-sale/listing/1FMCU0JD2HUD81465/2017-ford-escape/
https://www.truecar.com/used-cars-for-sale/listing/SAJWA0E72D8S96922/2013-jaguar-xf/
https://www.truecar.com/used-cars-for-s

https://www.truecar.com/used-cars-for-sale/listing/1G1ZD5ST8JF178083/2018-chevrolet-malibu/
https://www.truecar.com/used-cars-for-sale/listing/3FAFP11392R226759/2002-ford-escort/
https://www.truecar.com/used-cars-for-sale/listing/2GNAXPEX3K6282621/2019-chevrolet-equinox/
https://www.truecar.com/used-cars-for-sale/listing/1FAFP53S6XA214453/1999-ford-taurus/
https://www.truecar.com/used-cars-for-sale/listing/1FMCU0GD2HUA94621/2017-ford-escape/
https://www.truecar.com/used-cars-for-sale/listing/5UXWX9C56H0T12738/2017-bmw-x3/
https://www.truecar.com/used-cars-for-sale/listing/3FA6P0HD0HR214419/2017-ford-fusion/
https://www.truecar.com/used-cars-for-sale/listing/JTHBN30F230102922/2003-lexus-ls/
https://www.truecar.com/used-cars-for-sale/listing/3GCPCSE03DG372802/2013-chevrolet-silverado-1500/
https://www.truecar.com/used-cars-for-sale/listing/1FDWX36P27EB32276/2007-ford-super-duty-f-350-chassis-cab/
https://www.truecar.com/used-cars-for-sale/listing/5UXWX9C37H0T17350/2017-bmw-x3/
https://ww

...

https://www.truecar.com/used-cars-for-sale/listing/2T1BURHE4KC148856/2019-toyota-corolla/
https://www.truecar.com/used-cars-for-sale/listing/3VWBP7AJ5CM124235/2012-volkswagen-jetta/
https://www.truecar.com/used-cars-for-sale/listing/JTDKARFU9H3537707/2017-toyota-prius/
https://www.truecar.com/used-cars-for-sale/listing/1FTFX1CF5DFD81729/2013-ford-f-150/
https://www.truecar.com/used-cars-for-sale/listing/1FTEW1EG7HKD58468/2017-ford-f-150/
https://www.truecar.com/used-cars-for-sale/listing/1C6RR7KG8ES269849/2014-ram-1500/
https://www.truecar.com/used-cars-for-sale/listing/1C6SRFGT5KN613925/2019-ram-1500/
https://www.truecar.com/used-cars-for-sale/listing/3C4NJCCBXKT802403/2019-jeep-compass/
https://www.truecar.com/used-cars-for-sale/listing/3N1CN7AP3JL800883/2018-nissan-versa/
https://www.truecar.com/used-cars-for-sale/listing/5NPE24AF3KH751741/2019-hyundai-sonata/
https://www.truecar.com/used-cars-for-sale/listing/5NPD84LF8JH301606/2018-hyundai-elantra/
https://www.truecar.com/used-cars

https://www.truecar.com/used-cars-for-sale/listing/4T1B11HK3JU595966/2018-toyota-camry/
https://www.truecar.com/used-cars-for-sale/listing/3FA6P0H7XHR108778/2017-ford-fusion/
https://www.truecar.com/used-cars-for-sale/listing/3N1CN7AP9JL846444/2018-nissan-versa/
https://www.truecar.com/used-cars-for-sale/listing/1FTBF2A65GEA38634/2016-ford-super-duty-f-250/
https://www.truecar.com/used-cars-for-sale/listing/3MZBN1L32JM202582/2018-mazda-mazda3/
https://www.truecar.com/used-cars-for-sale/listing/3C4PDCEG1JT234128/2018-dodge-journey/
https://www.truecar.com/used-cars-for-sale/listing/WBAYM1C54DDZ02621/2013-bmw-6-series/
https://www.truecar.com/used-cars-for-sale/listing/5J6RE3H71AL031442/2010-honda-cr-v/
https://www.truecar.com/used-cars-for-sale/listing/4T1B11HK3JU543544/2018-toyota-camry/
https://www.truecar.com/used-cars-for-sale/listing/KNDJP3A53J7578775/2018-kia-soul/
https://www.truecar.com/used-cars-for-sale/listing/1GC1KVEG1GF236942/2016-chevrolet-silverado-2500hd/
https://www.tru

Process ForkPoolWorker-182:
Process ForkPoolWorker-185:
Process ForkPoolWorker-186:
Process ForkPoolWorker-183:
Process ForkPoolWorker-180:
Process ForkPoolWorker-181:
Process ForkPoolWorker-173:
Process ForkPoolWorker-184:


KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/jp/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/jp/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/jp/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/jp/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/jp/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/jp/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/jp/anaconda3/lib/python3.7/multiprocessing/pr

In [60]:
print(data.shape)
data.head(20)

(9841, 19)


,year,make,model,sub_model,city,state,mileage,price,exterior_color,interior_color,mpg_city,mpg_hwy,engine,transmission,drive_type,fuel_type,popular_feature,vehicle_history,cpo
0,2017,Ford,Fusion,SE FWD,Blue Mound,TX,"32,253","$14,400",White Platinum Metallic Tri-Coat,Medium Light Stone,21,32,1.5L Inline-4 Gas Turbocharged,Automatic,FWD,Gas,"[Navigation, Backup Camera, Cruise Control, Bl...","[0, reported accidents, Fleet or Mixed Use, C...",False
1,2007,BMW,6,650i Convertible,Stanton,CA,"59,664","$11,499",Mineral Silver Metallic,Tan,16,22,4.8L V-8 Gas,Automatic,RWD,Gas,[],"[0, reported accidents, Fleet or Mixed Use, C...",False
2,2019,Kia,Optima,LX,Thornton,CO,"4,816","$18,586",Ebony Black,Black,24,33,2.4L Inline-4 Gas,Automatic,FWD,Gas,"[Cruise Control, Lane Keep Assist, Bluetooth, ...","[1, reported accidents, Personal or Rental Us...",True
3,2017,Ford,Fusion,SE FWD,Denver,NC,"99,515","$8,999",Shadow Black,Black,21,32,2.5L Inline-4 Gas,Automatic,FWD,Gas,"[Cruise Control, Bluetooth, Backup Camera]","[1, reported accidents, Fleet or Mixed Use, C...",False
4,2017,Hyundai,Elantra,SE 2.0L Sedan Automatic (alt),Wesley Chapel,FL,"117,200","$6,109",Quartz White Pearl,Beige,29,38,2.0L Inline-4 Gas,Automatic,FWD,Gas,"[Cruise Control, Bluetooth]","[1, reported accidents, Personal or Rental Us...",False
5,2017,Honda,Civic,LX Hatchback CVT,Duluth,GA,"102,068","$9,985",Crystal Black Pearl,Select,31,40,1.5L Inline-4 Gas Turbocharged,Automatic,FWD,Gas,"[Cruise Control, Bluetooth, Backup Camera]","[2, reported accidents, Fleet or Mixed Use, C...",False
6,2016,Ford,F-150,XLT SuperCrew 5.5' Box RWD,Murrieta,CA,"22,650","$26,132",Shadow Black,Black,18,25,3.5L V-6 Gas Turbocharged,Automatic,RWD,Gas,"[Cruise Control, Bluetooth]","[0, reported accidents, Personal or Rental Us...",False
7,2016,Ford,F-150,XLT SuperCrew 5.5' Box 4WD,Sacramento,CA,"103,293","$22,997",White,Gray,17,23,5.0L V-8 Gas,Automatic,4WD,Gas,[],"[0, reported accidents, Personal or Rental Us...",False
8,2017,Nissan,Rogue,SV AWD,Palmetto Bay,FL,"78,116","$9,997",Magnetic Black,Charcoal,25,32,2.5L Inline-4 Gas,Automatic,AWD,Gas,"[Front Heated Seats, Cruise Control, Backup Ca...","[2, reported accidents, Fleet or Mixed Use, C...",False
9,2017,Ford,Fusion,SE FWD,Houston,TX,"48,524","$10,466",Shadow Black,Unknown,21,32,2.5L Inline-4 Gas,Automatic,FWD,Gas,"[Cruise Control, Bluetooth, Backup Camera]","[0, reported accidents, Fleet or Mixed Use, C...",False


In [61]:
data.to_csv('usedCarListing.csv', encoding = 'utf-8')